(application)=
# Application runtime

You can use the {py:meth}`~mlrun.runtimes.ApplicationRuntime` to provide an image that runs on top of your deployed model. 

The application runtime deploys a container image (for example, a web application) that is exposed on a specific port, and a command to run the HTTP server. The runtime is based on top of Nuclio, and adds the application as a side-car to a Nuclio function pod while the actual function is a reverse proxy to that application. 

You can set an existing image to run in the application, or let the application runtime build the side-car image for you, by specifying the source code, and pulling at build-time. 

> Note: The default base image is `python:3.9` when not specifying and image for the application.
    
An API Gateway, by default, is in front of the application and can provide different authentication methods, or none.

Typical use cases are:
- Deploy a [Vizro](https://github.com/mckinsey/vizro) dashboard that communicates with an external source (for example, a serving model) to display graphs, data, and inference.
- Deploy a model and a UI &mdash; the model serving is the backend and the UI is the side car.
- Deploy a fastapi web-server with an MLRun model. In this case, the Nuclio function is a reverse proxy and the user web-app is the side car.


## Usage examples

Deploy a Vizro dashboard from a pre-built image:

In [ ]:
# Create an application runtime (with pre-built image)
application = project.set_function(
    name="my-vizro-dashboard", kind="application", image="repo/my-vizro-image:latest"
)
# Set the port that the side-car listens on
application.set_internal_application_port(port=8050)

# Deploy
application.deploy()

Deploy a Vizro dashboard from a source archive or git:

In [ ]:
# Specify the source to be loaded at build-time or run-time
application = project.set_function(
    name="my-vizro-dashboard", kind="application", requirements=["vizro"]
)
application.set_internal_application_port(8050)
application.spec.command = "gunicorn"
application.spec.args = [
    "<my-app>:<my-server>",
    "--bind",
    "0.0.0.0:8050",
]

# Provide code artifacts
application.with_source_archive(
    "git://github.com/org/repo#my-branch", pull_at_runtime=False
)

# Build the application image via MLRun and deploy the Nuclio function
# Optionally add mlrun
application.deploy(with_mlrun=False)

Reusing an already built reverse proxy image is done when:
- Redeploying a function that built the reverse proxy once and has `application.status.container_image` enriched.
- It was already built manually with `mlrun.runtimes.ApplicationRuntime.deploy_reverse_proxy_image()`.

Using one of the above options can help minimize redundant builds and streamline the development cycle. 

### Authentication modes

An application runtime can be accessed through an API Gateway that supports various authentication methods. 

The default authentication mode is `none` for open source and `access-key` for the Iguazio platform.

The different authentication modes can be configured as follows:

In [ ]:
from mlrun.common.schemas.api_gateway import APIGatewayAuthenticationMode

# Deploy without authentication
application.deploy(
    authentication_mode=APIGatewayAuthenticationMode.none,
)

# Basic authentication mode.
# This means that the application can be invoked only using the provided credentials
application.deploy(
    authentication_mode=APIGatewayAuthenticationMode.basic,
    authentication_creds=("my-username", "my-password"),
)

# Access-key authentication mode. the application can be invoked only with a valid session
application.deploy(
    authentication_mode=APIGatewayAuthenticationMode.access_key,
)

### Application runtime in a dark environment

To use application runtime in a dark (air-gapped) environment, you need to build the reverse proxy image and push it to a private registry, following the steps below:

In [ ]:
# 1. Create the reverse proxy image in a non air-gapped system
import mlrun.runtimes

mlrun.runtimes.ApplicationRuntime.deploy_reverse_proxy_image()

# 2. The created image name is saved on the ApplicationRuntime class:
mlrun.runtimes.ApplicationRuntime.reverse_proxy_image

# 3. Push the created image to the system’s docker registry

# 4. On the air-gapped environment, set the image on the class property:
mlrun.runtimes.ApplicationRuntime.reverse_proxy_image = (
    "registry/reverse-proxy-image:<tag>"
)

# 5. When creating application functions, this image will be used as the reverse proxy image, and it won’t be built again.